<a href="https://colab.research.google.com/github/estern95/Generative-Models/blob/main/text_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import requisite libraries. 

In [ ]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint



NameError: ignored

# Load Text

In [ ]:
!curl -s https://www.gutenberg.org/files/84/84-0.txt -o "frankenstein.txt"

file = open("frankenstein.txt").read()


# Perform Transforms

In [ ]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

processed_inputs = tokenize_words(file)

# Create Embeddings


In [ ]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))


input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 270250
Total vocab: 43


# Define Length of Generated Text

In [ ]:
seq_length = 100
x_data = []
y_data = []

# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

# Check Number of Sequences

In [ ]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 270150


# Convert to Tensor Form


In [ ]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

y = np_utils.to_categorical(y_data)

# Define Model Architecture

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

## Compile the Model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Training

In [ ]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
1056/1056 [==============================] - ETA: 0s - loss: 2.9381
Epoch 00001: loss improved from inf to 2.93809, saving model to model_weights_saved.hdf5
1056/1056 [==============================] - 3894s 4s/step - loss: 2.9381
Epoch 2/4
1056/1056 [==============================] - ETA: 0s - loss: 2.7465
Epoch 00002: loss improved from 2.93809 to 2.74649, saving model to model_weights_saved.hdf5
1056/1056 [==============================] - 3875s 4s/step - loss: 2.7465
Epoch 3/4
 805/1056 [=====================>........] - ETA: 15:22 - loss: 2.5719

Load the trained model:

In [ ]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Generate a random seed to start our text generation!

In [ ]:
num_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Now take a look at what the model generated!

In [ ]:
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]